In [79]:
import numpy as np
import pandas as pd

# Preparación de Subset para entrenamiento de algoritmos de ML

In [80]:
import os

COVID_URL = os.path.join('..', 'datasets', 'Clean_Covid_Dataset.csv')
print(COVID_URL)

..\datasets\Clean_Covid_Dataset.csv


In [81]:
# Definir el tipo de dato hace el proceso más eficiente
types = {
    'SECTOR': np.int8,
    'ENTIDAD_UM': np.int8,
    'SEXO': np.int8,
    'PAC_HOSPITALIZADO': np.int8,
    'FECHA_INGRESO': 'object',
    'FECHA_SINTOMAS': 'object',
    'FECHA_DEF': 'object',
    'INTUBADO': np.int8,
    'NEUMONIA': np.int8,
    'EDAD': np.int8,
    'EMBARAZO': np.int8,
    'DIABETES': np.int8,
    'EPOC': np.int8,
    'ASMA': np.int8,
    'INMUSUPR': np.int8,
    'HIPERTENSION':np.int8,
    'CARDIOVASCULAR': np.int8,
    'OBESIDAD': np.int8,
    'RENAL_CRONICA': np.int8,
    'TABAQUISMO': np.int8,
    'UCI': np.int8
}

In [82]:
# 'latin' porque contiene acentos
df = pd.read_csv(COVID_URL, encoding='latin', dtype=types, index_col=0)

C:\Users\Sebastián\AppData\Roaming\Python\Python38\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [83]:
df.head(3)

,SECTOR,ENTIDAD_UM,SEXO,PAC_HOSPITALIZADO,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,INTUBADO,NEUMONIA,EDAD,...,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,UCI
0,12,9,0,0,2020-10-16,2020-10-16,9999-99-99,0,0,41,...,0,0,0,0,0,0,0,0,0,0
1,12,23,1,1,2020-07-20,2020-07-17,2020-07-21,1,1,66,...,1,0,0,0,1,0,1,0,0,1
8,12,12,0,0,2020-08-19,2020-08-17,9999-99-99,0,0,47,...,1,0,0,0,0,0,0,0,0,0


In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3667730 entries, 0 to 10785803
Data columns (total 21 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   SECTOR             int8  
 1   ENTIDAD_UM         int8  
 2   SEXO               int8  
 3   PAC_HOSPITALIZADO  int8  
 4   FECHA_INGRESO      object
 5   FECHA_SINTOMAS     object
 6   FECHA_DEF          object
 7   INTUBADO           int8  
 8   NEUMONIA           int8  
 9   EDAD               int8  
 10  EMBARAZO           int8  
 11  DIABETES           int8  
 12  EPOC               int8  
 13  ASMA               int8  
 14  INMUSUPR           int8  
 15  HIPERTENSION       int8  
 16  CARDIOVASCULAR     int8  
 17  OBESIDAD           int8  
 18  RENAL_CRONICA      int8  
 19  TABAQUISMO         int8  
 20  UCI                int8  
dtypes: int8(18), object(3)
memory usage: 174.9+ MB


## Estrategia

Vamos a entrenar dos modelos de aprendizaje automático para:
1. Detectar si una persona morirá por COVID-19, tomando en cuenta las afeccione sque padece además de otra información médica y
2. En caso de resultar en un pronóstico positivo, pronosticar cuánto tiempo de vida le queda

Estos dos modelos predictivos ayudarán en el diagnóstico médico de nuevos pacientes que ingresan a hospitales todos los días, reduciendo la carga de trbajo del personal médico y permitíendoles detectar con mayor facilidad a los pacientes que requiere cuidado prioritario.

Los atributos que se usarán para entrenar los modelos son:
- Edad
- Si es paciente hospitalizado o ambulatorio
- Afecciones previas con las que cuenta
- Días que han pasado desde que presentó los síntomas hasta que fue hospitalizado
   
Obviamente, debido a que entrenaremos dos modelos, uno de clasificación y otro de regresión, necesitamos dos etiquetas diferentes. Estas etiquetas son:

- Si la personas falleció o no
- Días que pasaron desde que la persona fue hospitalizado hasta que falleció

Los dos subsets de datos compartirán las mismas características, pero un subset tendrá la primera etiqueta mientras que el otro subset se que da con la etiqueta restante

## Selección de Datos
Cuatro columnas no son de nuestro interés en ninguno de los modelos: SECTOR, ENTIDAD_UM, INTUBADO y UCI por lo que se eliminarán

In [85]:
df.drop(['SECTOR', 'ENTIDAD_UM', 'INTUBADO', 'UCI'], axis=1, inplace=True)

In [86]:
df.columns

Index(['SEXO', 'PAC_HOSPITALIZADO', 'FECHA_INGRESO', 'FECHA_SINTOMAS',
       'FECHA_DEF', 'NEUMONIA', 'EDAD', 'EMBARAZO', 'DIABETES', 'EPOC', 'ASMA',
       'INMUSUPR', 'HIPERTENSION', 'CARDIOVASCULAR', 'OBESIDAD',
       'RENAL_CRONICA', 'TABAQUISMO'],
      dtype='object')

## Transformación de Datos
Ambos datasets requieren de un campo que cuente los días desde que el paciente presentó síntomas hasta que ingresó a la unidad médica. Para ello realizaremos operaciones entre fecha.


In [87]:
import datetime

In [88]:
def dias_desde_sintomas_a_ingreso(x):
    fecha_sintomas = datetime.date.fromisoformat(x['FECHA_SINTOMAS'])
    fecha_ingreso = datetime.date.fromisoformat(x['FECHA_INGRESO'])
    return (fecha_ingreso - fecha_sintomas).days

In [89]:
df['DIAS_SINTOMAS'] = df[['FECHA_SINTOMAS', 'FECHA_INGRESO']].apply(dias_desde_sintomas_a_ingreso, axis=1).astype('int8')

In [90]:
df.head(3)

,SEXO,PAC_HOSPITALIZADO,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,NEUMONIA,EDAD,EMBARAZO,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,DIAS_SINTOMAS
0,0,0,2020-10-16,2020-10-16,9999-99-99,0,41,0,0,0,0,0,0,0,0,0,0,0
1,1,1,2020-07-20,2020-07-17,2020-07-21,1,66,0,1,0,0,0,1,0,1,0,0,3
8,0,0,2020-08-19,2020-08-17,9999-99-99,0,47,0,1,0,0,0,0,0,0,0,0,2


In [91]:
# La columna FECHA_SINTOMAS ya no nos servirá, por lo que la desechamos
df.drop(['FECHA_SINTOMAS'], axis=1, inplace=True)

In [92]:
df.columns

Index(['SEXO', 'PAC_HOSPITALIZADO', 'FECHA_INGRESO', 'FECHA_DEF', 'NEUMONIA',
       'EDAD', 'EMBARAZO', 'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR',
       'HIPERTENSION', 'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA',
       'TABAQUISMO', 'DIAS_SINTOMAS'],
      dtype='object')

### Separación de Datos
Ya que a partir de ahora trabajaremos en dos datasets diferentes, los separaremos para mayor comodidad. El dataset usado para clasificación usará todos los datos que tenemos hasta ahora, pero el dataset para regresión usará sólo los registros de las personas fallecidas (aquellas cuya fecha de defuncion no sea 9999-99-99)

In [93]:
df_fallecido = df
df_dias = df[df['FECHA_DEF']!='9999-99-99'].copy()

In [94]:
# Registros del dataset de clasificación
df_fallecido.shape[0]

3667730

In [95]:
# Registros del dataset de regresión
df_dias.shape[0]

275496

Para generar las etiquetas de los subssets de entrenamiento, necesitamos operar con los campos de FECHA. En el caso de lla etiqueta de clasificación, generarla es sencilla.   
Para la etiqueta de regresión, tendremos que convertir la fecha a un tipo de dato correcto, y luego hacer cálculos entre fechas

### Generación de Etiqueta de Clasificación
En este caso, la etiqueta es facilmente generada consultado el campo FECHA_DEF en busca de fechas válidad que indiquen sia gluien falleció.

In [96]:
df['FALLECIDO'] = (df_fallecido['FECHA_DEF']!='9999-99-99').astype('int8')

In [97]:
df_fallecido.head(3)

,SEXO,PAC_HOSPITALIZADO,FECHA_INGRESO,FECHA_DEF,NEUMONIA,EDAD,EMBARAZO,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,DIAS_SINTOMAS,FALLECIDO
0,0,0,2020-10-16,9999-99-99,0,41,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,2020-07-20,2020-07-21,1,66,0,1,0,0,0,1,0,1,0,0,3,1
8,0,0,2020-08-19,9999-99-99,0,47,0,1,0,0,0,0,0,0,0,0,2,0


Una vez generados los datos, eliminamos columnas innecesarias (fechas)

In [98]:
df_fallecido.drop(['FECHA_INGRESO', 'FECHA_DEF'], axis=1, inplace=True)

In [99]:
df_fallecido.columns

Index(['SEXO', 'PAC_HOSPITALIZADO', 'NEUMONIA', 'EDAD', 'EMBARAZO', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'CARDIOVASCULAR',
       'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO', 'DIAS_SINTOMAS',
       'FALLECIDO'],
      dtype='object')

### Generación de Etiqueta de Regresión
En este caso, el procedimiento involucra operaciones entre fechas, más específicamente una resta entre FECHA_DEF y FECHA_INGRESO, en días.

In [100]:
def dias_desde_ingreso_a_defuncion(x):
    fecha_defuncion = datetime.date.fromisoformat(x['FECHA_DEF'])
    fecha_ingreso = datetime.date.fromisoformat(x['FECHA_INGRESO'])
    return (fecha_defuncion - fecha_ingreso).days

In [101]:
df_dias['DIAS_DEF'] = df_dias[['FECHA_INGRESO', 'FECHA_DEF']].apply(dias_desde_ingreso_a_defuncion, axis=1).astype('int8')

In [102]:
df_dias.head(3)

,SEXO,PAC_HOSPITALIZADO,FECHA_INGRESO,FECHA_DEF,NEUMONIA,EDAD,EMBARAZO,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,DIAS_SINTOMAS,DIAS_DEF
1,1,1,2020-07-20,2020-07-21,1,66,0,1,0,0,0,1,0,1,0,0,3,1
152,0,1,2020-12-14,2020-12-17,1,58,0,1,0,0,0,1,1,0,1,0,2,3
174,1,1,2020-12-15,2020-12-15,1,30,0,0,0,0,0,0,0,0,1,0,0,0


Una vez generados los datos, eliminamos columnas innecesarias

In [103]:
df_dias.drop(['FECHA_INGRESO', 'FECHA_DEF'], axis=1, inplace=True)

In [104]:
df_dias.columns

Index(['SEXO', 'PAC_HOSPITALIZADO', 'NEUMONIA', 'EDAD', 'EMBARAZO', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'CARDIOVASCULAR',
       'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO', 'DIAS_SINTOMAS', 'DIAS_DEF'],
      dtype='object')

In [106]:
df_dias

,SEXO,PAC_HOSPITALIZADO,NEUMONIA,EDAD,EMBARAZO,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,DIAS_SINTOMAS,DIAS_DEF
1,1,1,1,66,0,1,0,0,0,1,0,1,0,0,3,1
152,0,1,1,58,0,1,0,0,0,1,1,0,1,0,2,3
174,1,1,1,30,0,0,0,0,0,0,0,0,1,0,0,0
25659,0,1,1,47,0,1,0,0,0,1,0,0,0,0,0,12
25875,1,1,1,83,0,1,1,1,0,1,1,0,0,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10785082,0,1,1,73,0,0,0,0,0,0,0,0,0,0,10,14
10785091,0,1,1,87,0,1,0,0,0,1,0,0,0,0,3,8
10785097,1,1,1,77,0,0,1,0,0,1,0,0,0,0,7,11
10785105,0,1,1,58,0,1,0,0,0,0,0,0,0,0,13,4


## Salvamos los datos
Una vez generados los subsets de datos, solo habrá que salvarlos en dos archivos diferentes

In [107]:
CLASIF_SUBSET_PATH = os.path.join('..', 'datasets', 'subset_clasif.csv')
REGRES_SUBSET_PATH = os.path.join('..', 'datasets', 'subset_regres.csv')

In [109]:
df_fallecido.to_csv(CLASIF_SUBSET_PATH)

In [ ]:
df_dias.to_csv(REGRES_SUBSET_PATH)